In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [2]:
df = pd.read_csv('../data/DF_modelos/df_industrial_total.csv')

In [3]:
df.head()

## Paso 1: Preparación de los datos.

In [4]:
df.dtypes

In [5]:
df.nunique()

In [6]:
df.shape

In [7]:
# Convertir la columna de fecha a formato datetime
df['fecha'] = pd.to_datetime(df['fecha'])

In [8]:
df['Sector_Economic'] = df['Sector_Economic'].astype('category')

### *Transformación variable `Fecha`*

1. Convertir fechas a formato numérico (timestamp)

In [9]:
# Asegúrate de que la columna de fecha está en formato datetime
df['fecha'] = pd.to_datetime(df['fecha'], format='%Y')  # Ajusta el formato según tus datos

# Convertir la columna de fecha a timestamp
df['fecha'] = df['fecha'].astype(np.int64) // 10**9  # Convertir a dias


In [10]:
#Normalizar las columnas numéricas
scaler = StandardScaler()
df[['consumo', 'tmed', 'prec', 'velmedia', 'poblacion', 'PIB']] = scaler.fit_transform(df[['consumo', 'tmed', 'prec', 'velmedia', 'poblacion', 'PIB']])


In [11]:
skew = df.skew(axis=0, numeric_only=True)
skew

In [12]:
my_list = []
for key,value in enumerate(skew):
    if value > 1:
        my_dict = {key:value}
        my_list.append(my_dict)
        
print(my_list)


## Paso 2:  Definición de la arquitectura de la red neuronal

1. Dividir los datos en características y objetivo

In [13]:
# Definir características (X) y objetivo (y)
X = df.drop(columns=['consumo', 'Sector_Economic'])  
y = df['consumo']  

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

2. Construir el modelo de red neuronal

In [14]:
# Inicializar el modelo
modelInd = Sequential()

# Añadir la capa de entrada
modelInd.add(Input(shape=(X_train.shape[1],)))  # Define la forma de entrada aquí

# Añadir la primera capa oculta
modelInd.add(Dense(units=64, activation='relu'))

# Añadir una segunda capa oculta (opcional)
modelInd.add(Dense(units=32, activation='relu'))

# Capa de salida
modelInd.add(Dense(units=1))  # Para una regresión, una sola neurona sin función de activación

# Compilar el modelo
modelInd.compile(optimizer='adam', loss='mean_squared_error')


3. Entrenar el modelo

In [15]:
# Entrenar el modelo
history = modelInd.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)


4. Evaluar el modelo

In [16]:
test_loss = modelInd.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

## Paso 3: Evaluación del modelo y predicciones

1. Visualizar la pérdida durante el entrenamiento

In [17]:
# Visualizar la pérdida durante el entrenamiento
plt.plot(history.history['loss'], label='Pérdida en entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida en validación')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()

2. Evaluar el modelo en el conjunto de prueba

In [18]:
# Evaluar el modelo en el conjunto de prueba
test_loss = modelInd.evaluate(X_test, y_test)
print(f'Pérdida en el conjunto de prueba: {test_loss}')

3. Hacer predicciones

In [19]:
# Hacer predicciones
predicciones = modelInd.predict(X_test)

# Comparar predicciones con los valores reales
plt.scatter(y_test, predicciones)
plt.xlabel('Valores reales')
plt.ylabel('Predicciones')
plt.title('Comparación de valores reales vs. predicciones')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)  # Línea de identidad
plt.show()

4. Calcular métricas adicionales

In [20]:
mae = mean_absolute_error(y_test, predicciones)
rmse = mean_squared_error(y_test, predicciones, squared=False)

print(f'Error Absoluto Medio (MAE): {mae}')
print(f'Raíz del Error Cuadrático Medio (RMSE): {rmse}')


In [21]:
#Extraccion del modelo a una crapeta
# Guardar el modelo entrenado en una carpeta
modelInd.save('../models/modelo_industrial.h5')


## Paso 4: Optimización del modelo

Aquí hay algunas estrategias que puedes utilizar para optimizar tu modelo:

#### 1. **Ajuste de Hiperparámetros**

- **Número de Neuronas y Capas**: Experimenta con el número de neuronas en cada capa y la cantidad de capas ocultas. Un modelo más complejo puede capturar mejor patrones en los datos, pero ten cuidado con el sobreajuste.

- **Tasa de Aprendizaje (Learning Rate)**: Ajusta la tasa de aprendizaje del optimizador. Puedes utilizar un optimizador como Adam que permite cambiar dinámicamente la tasa de aprendizaje.


### 2. **Regularización**

- **Dropout**: Introduce capas de `Dropout` para evitar el sobreajuste. Esto apaga aleatoriamente un porcentaje de neuronas durante el entrenamiento, lo que ayuda a que el modelo generalice mejor
- **Regularización L1 o L2**: Aplica técnicas de regularización a las capas densas.


#### 3. **Aumentar el número de épocas**

Aumentar el número de épocas puede ayudar a que el modelo aprenda mejor, pero asegúrate de monitorear la pérdida en el conjunto de validación para evitar el sobreajuste.


#### 4. **Early Stopping**

Implementa `EarlyStopping` para detener el entrenamiento si la pérdida en el conjunto de validación no mejora después de un número específico de épocas. Esto ayuda a evitar el sobreajuste.


#### 5. **Prueba con diferentes optimizadores**

Prueba diferentes optimizadores (como SGD, Adam, RMSprop) para ver si alguno mejora el rendimiento.


### 6. **Evaluar mejoras**

Después de aplicar algunas de estas técnicas, vuelve a evaluar el modelo utilizando el conjunto de prueba y compara el nuevo `Test Loss` y el `RMSE` con los resultados anteriores.
